<a href="https://colab.research.google.com/github/jayeshbhole/YoloV3-Tiny-Google-Colab/blob/master/Train_Tiny_YoloV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Enabling GPU within your notebook

You need to enable GPU acceleration within your Colab notebook so that your YOLOv3 system will be able to process detections over 100 faster than CPU.

Steps:

    i) Click Edit at top left of your notebook

    ii) Click Notebook Settings within dropdown

    iii) Under 'Hardware Accelerator' select GPU and then hit Save

    iv) Set the Runtime type to Python 3


Your notebook should now have GPU enabled!

#Step 2 : Mounting the Google Drive

Mount the Google Drive to your runtime by executing the following commands.

The cell will give a link as output. Open it and login to your Google Account. Copy the authorisation code and paste it in the box and hit enter.

In [0]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

----
Here we create a symbolic link for the drive ie. the paths are equivalent
 
    '/content/gdrive/My\ Drive/' is equivalent to '/mydrive 


In [0]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
#if your drive cannot be linked, uncomment and run the next command
#!sudo unlink /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

#Step 3 : Cloning Darknet and Building it

The following cells will clone darknet from [AlexeyAB's darknet repository](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects) , adjust the Makefile to enable OPENCV and GPU for darknet and then build darknet.

Ignore warnings when you run the '!make' cell!

In [0]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [0]:
# make darknet (build)
!make

#Step 4 : Download the pretrained weights for Yolo

In [0]:
# get yolov3-tiny pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

      #OR
      
#!cp /mydrive/yolov3-tiny.weights /darknet

In [0]:
!./darknet partial cfg/yolov3-tiny.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15

#Step 5:
###Define general functions

In [0]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

###Run Detection to check if everything is working fine

In [0]:
# look we can run detection on tiny yolo
!./darknet detect cfg/yolov3-tiny.cfg yolov3-tiny.weights data/dog.jpg
imShow('predictions.jpg')

#Step 6 :
##Move your training images to the data folder(only if the images are few)


In [0]:
#!cd ../data
# !mkdir obj
#!cp -r /mydrive/yolotiny/obj ../data/obj

## obj.names and obj.data

Create a new file within a code or text editor called obj.names and you will make this file exactly the same as your classes.txt in the dataset generation step.


You will also create a obj.data file and fill it in like this (change your number of classes accordingly, as well as your backup location). 
This backup path is where we will save the weights to of our model throughout training. Create a backup folder in your google drive and put its correct path in this file.

## train.txt
This file will contain the relative paths of all your training images.

In [0]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolo/obj.names ./data
!cp /mydrive/yolo/obj.data  ./data
!cp /mydrive/yolo/train.txt  ./data

# upload the obj.names and obj.data files to cloud VM from local machine (uncomment to use)
#%cd data
#upload()
#%cd .. 

## .cfg file
Make changes to the cfg file as per the git repo of [AlexeyAB](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects) and upload it to the Cloud VM.

In [0]:
#copy the cfg file 
!cp /mydrive/yolotiny/yolov3-tiny-obj.cfg ../darknet/cfg

If you have saved the weights file in Drive, copy it in the darknet directory. 

In [0]:
# upload pretrained convolutional layer weights through drive
#!cp /mydrive/yolotiny/yolov3-tiny.conv.15 ../darknet

#Final Step :
Hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Add the following code to your console window and hit enter.
```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```
Now, it will click the screen every 10 minutes so that you don't get kicked off for being idle.

Start your training.



In [0]:
!./darknet detector train data/obj.data yolov3-tiny-obj.cfg yolov3-tiny.conv.15 -dont_show
imShow('chart.png')

In [0]:
#Re train using past weights
!./darknet detector train data/obj.data cfg/yolov3-tiny-obj.cfg /mydrive/yolotiny/backup/yolov3-tiny-obj_last.weights -dont_show
imShow('chart.png')
#download('chart.png')

##Test your model :
Set the cfg file to training parameters and run the detector.

In [0]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3-tiny-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3-tiny-obj.cfg
%cd ..

In [0]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov3-tiny-obj.cfg /mydrive/yolotiny/backup/yolov3-tiny-obj_5000.weights /mydrive/yolotiny/obj/77.JPEG -thresh 0.3
imShow('predictions.jpg')